In [61]:
import daw_readout
import process_data
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import sys
import time
import os

In [62]:
#file_list = 'runlist/LED_20240826_1.7V_17ns'
file_list = 'runlist/runlist'

In [63]:
path_save =""
winfo =[]
with open(file_list, 'r') as list:
    for line in list:    
        rawfilename = line.rstrip('\n')#[17 :] 
        #file_tag = line.rstrip('\n').rstrip('.bin')[24:][: -12] 
        #print(file_tag)
        #break   
        rawdata = daw_readout.DAWDemoWaveParser(rawfilename)     
        for wave in tqdm(rawdata) :            
            ch = wave.Channel
            ttt = wave.Timestamp
            base = wave.Baseline
            pulse = wave.Waveform
            st, minp, ed = process_data.pusle_index(pulse)
            ht = base - wave.Waveform[minp]          
            area = process_data.pulse_area(pulse, st, ed, base)
            area_fix_range = process_data.pulse_area_fix_range(pulse, 90, 250, base)
            area_fix_range_dy = process_data.pulse_area_fix_range(pulse, 90, 250, base)
            if ch == 0:
                ht = ht * 2.76
                area_pe = area / 77.791 *2.76
                area_fix_range_pe = area_fix_range / 77.791 *2.76
            if ch == 1:
                area_pe = area / 96.99
                area_fix_range_pe = area_fix_range / 96.99
            if ch == 2:
                area_pe = area / 77.791
                area_fix_range_pe = area_fix_range_dy / 77.791 
            winfo.append({         
                #'EvtID': i,
                'Ch':ch,
                'TTT':ttt,   ## Trigger time tag
                'Baseline': base, 
                'Hight': ht, 
                'St': st,
                'Ed': ed,
                'Minp': minp,
                'Width': ed - st,  
                'Area': area_pe,
                'Area_fixrange':area_fix_range_pe,
                'Wave': pulse
            })            
        #file_tag = line.rstrip('\n')[17 :].rstrip('.bin')[24:][: -12]  
        file_tag = line.rstrip('\n').rstrip('.bin')[24:][: -12]  
        path_save = "outnpy/{}.h5df".format(file_tag)
        df = pd.DataFrame(winfo)
        process_data.write_to_hdf5(df, path_save)

386004it [00:16, 22807.89it/s]


h5 Write Time: 2.00 s 
h5 File Size: 242.05 MB
Save to outnpy/lv2415_anode_lv2414_dual_20241011_LED_885mv_900mv_500ns_attenuator_9DB_400_ratio_run0.h5df


In [64]:
#df = pd.DataFrame(winfo)
#data_array = df.values
#np.save(path_save, data_array)